In [28]:
import pydot
from IPython.display import Image, display
import queue
import copy
import numpy as np

Podemos reusar la clase nodo y árbol desarrollada para los algoritmos de búsqueda a ciega y búsqueda heurística. Adicionando el método minimax y alfa-beta a la última

In [29]:
class Node ():
  def __init__(self, state,value,operators,operator=None, parent=None,objective=None):
    self.state= state
    self.value = value
    self.children = []
    self.parent=parent
    self.operator=operator
    self.objective=objective
    self.level=0
    self.operators=operators
    self.v=0


  def add_child(self, value, state, operator):
    node=type(self)(value=value, state=state, operator=operator,parent=self,operators=self.operators)
    node.level=node.parent.level+1
    self.children.append(node)
    return node

  def add_node_child(self, node):
    node.level=node.parent.level+1
    self.children.append(node)
    return node

  #Devuelve todos los estados según los operadores aplicados
  def getchildrens(self):
    return [
        self.getState(i)
          if not self.repeatStatePath(self.getState(i))
            else None for i, op in enumerate(self.operators)]

  def getState(self, index):
    pass

  def __eq__(self, other):
    return self.state == other.state

  def __lt__(self, other):
    return self.f() < other.f()


  def repeatStatePath(self, state):
      n=self
      while n is not None and n.state!=state:
          n=n.parent
      return n is not None

  def pathObjective(self):
      n=self
      result=[]
      while n is not None:
          result.append(n)
          n=n.parent
      return result

  def heuristic(self):
    return 0

  def cost(self):
    return 1

  def f(self):
    return self.cost()+self.heuristic()

  ### Crear método para criterio objetivo
  ### Por defecto vamos a poner que sea igual al estado objetivo, para cada caso se puede sobreescribir la función
  def isObjective(self):
    return (self.state==self.objetive.state)


In [30]:
class Tree ():
  def __init__(self, root ,operators):
    self.root=root
    self.operators=operators
    self.nodes_explored = 0  # Contador de nodos explorados

  def printPath(self,n):
    stack=n.pathObjective()
    path=stack.copy()
    while len(stack)!=0:
        node=stack.pop()
        if node.operator is not None:
            print(f'operador:  {self.operators[node.operator]} \t estado: {node.state}')
        else:
            print(f' {node.state}')
    return path

  def miniMax(self, depth):
    self.nodes_explored = 0  # Reiniciar contador
    is_max = self.root.player
    self.root.v = self.miniMaxR(self.root, depth, is_max)
    values = [c.v for c in self.root.children]
    if is_max:
        best = max(values)
    else:
        best = min(values)
    index = values.index(best)
    return self.root.children[index]

  def miniMaxR(self, node, depth, maxPlayer):
    self.nodes_explored += 1  # Incrementar contador
    if depth==0 or node.isObjective():
      node.v=node.heuristic()
      return node.heuristic()
    ## Generar los hijos del nodo
    children=node.getchildrens()

    ## Según el jugador que sea en el árbol
    if maxPlayer:
      value=float('-inf')
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node,
                                   operators=node.operators,player=False)
          newChild=node.add_node_child(newChild)
          value=max(value,self.miniMaxR(newChild,depth-1,False))
      #node.v=value
      #return value
    else:
      value=float('inf')
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node,
                                   operators=node.operators,player=True)
          newChild=node.add_node_child(newChild)
          value=min(value,self.miniMaxR(newChild,depth-1,True))
    node.v=value
    return value

  def alphaBeta(self, depth):
    self.nodes_explored = 0  # Reiniciar contador
    is_max = self.root.player
    self.root.v = self.alphaBetaR(self.root, depth, is_max, float('-inf'), float('inf'))
    values = [c.v for c in self.root.children]
    if is_max:
        best = max(values)
    else:
        best = min(values)
    index = values.index(best)
    return self.root.children[index]

  def alphaBetaR(self, node, depth, maxPlayer, a, b):
    self.nodes_explored += 1  # Incrementar contador

    if depth == 0 or node.isObjective():
        node.v = node.heuristic()
        return node.heuristic()

    children = node.getchildrens()

    if maxPlayer:
        value = float('-inf')
        for i, child in enumerate(children):
            if child is not None:
                newChild = type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node,
                    operators=node.operators,
                    player=False)

                newChild = node.add_node_child(newChild)
                valor_hijo = self.alphaBetaR(newChild, depth-1, False, a, b)
                value = max(value, valor_hijo)
                a = max(a, value)
                if a >= b:
                    break
    else:
        value = float('inf')
        for i, child in enumerate(children):
            if child is not None:
                newChild =type(self.root)(value=node.value + '-' + str(i),state=child,operator=i,parent=node,
                    operators=node.operators,
                    player=True)
                newChild = node.add_node_child(newChild)
                valor_hijo = self.alphaBetaR(newChild, depth-1, True, a, b)
                value = min(value, valor_hijo)
                b = min(b, value)
                if a >= b:
                    break
    node.v = value
    return value

In [31]:
class NodeIsolation(Node):
    """
    Juego Isolation en tablero 5x5.

    Estado: dict con:
        'board'      : lista 5x5  (1 = disponible, 0 = eliminada)
        'pos1'       : (fila, col) jugador Max / X
        'pos2'       : (fila, col) jugador Min / O
        'max_turn'   : bool — True si en este nodo le toca mover a Max

    CLAVE: el jugador activo se guarda en el ESTADO (no en self.player),
    porque el Tree sobreescribe self.player al crear los nodos hijos.
    """

    MOVES = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]

    def __init__(self, player=True, **kwargs):
        super().__init__(**kwargs)
        self.player = player
        self.v = float('-inf') if player else float('inf')

    # ── helpers ──────────────────────────────────────────────────────────────

    def _board(self):     return self.state['board']
    def _pos1(self):      return self.state['pos1']
    def _pos2(self):      return self.state['pos2']
    def _max_turn(self):  return self.state.get('max_turn', True)

    def _valid_moves(self, pos, board, other_pos=None):
        """Obtiene movimientos válidos evitando superposición con la otra ficha"""
        r, c = pos
        return [
            (r+dr, c+dc)
            for dr, dc in self.MOVES
            if (0 <= r+dr < 5 and 0 <= c+dc < 5 and
                board[r+dr][c+dc] == 1 and
                (other_pos is None or (r+dr, c+dc) != other_pos))
        ]

    def _removable_cells(self, board, pos1, pos2):
        return [
            (r, c)
            for r in range(5) for c in range(5)
            if board[r][c] == 1 and (r,c) != pos1 and (r,c) != pos2
        ]

    # ── getchildrens ──────────────────────────────────────────────────────────

    def getchildrens(self):
        """
        Genera hijos con turno completo (mover + eliminar).
        El jugador activo se lee de state['max_turn'], NO de self.player,
        porque el Tree machaca self.player al construir el árbol.
        """
        board  = self._board()
        pos1   = self._pos1()
        pos2   = self._pos2()
        is_max = self._max_turn()   # ← fuente de verdad del turno

        current = pos1 if is_max else pos2
        other = pos2 if is_max else pos1

        moves = self._valid_moves(current, board, other)
        if not moves:
            return []

        children_states = []
        for (nr, nc) in moves:
            new_pos1 = (nr, nc) if is_max else pos1
            new_pos2 = pos2     if is_max else (nr, nc)

            removable = self._removable_cells(board, new_pos1, new_pos2)

            if not removable:
                children_states.append({
                    'board':    [row[:] for row in board],
                    'pos1':     new_pos1,
                    'pos2':     new_pos2,
                    'max_turn': not is_max,
                })
            else:
                for (er, ec) in removable:
                    new_board = [row[:] for row in board]
                    new_board[er][ec] = 0
                    children_states.append({
                        'board':    new_board,
                        'pos1':     new_pos1,
                        'pos2':     new_pos2,
                        'max_turn': not is_max,
                    })

        return children_states

    def getState(self, index):
        return None

    # ── objetivo ─────────────────────────────────────────────────────────────

    def isObjective(self):
        """Terminal cuando el jugador activo (según el estado) no puede moverse."""
        board   = self._board()
        pos1    = self._pos1()
        pos2    = self._pos2()
        is_max  = self._max_turn()
        current = pos1 if is_max else pos2
        other = pos2 if is_max else pos1
        return len(self._valid_moves(current, board, other)) == 0

    # ── heuristic strategies ─────────────────────────────────────────────────

    def offensive(self, playerMoves, opponentMoves):
        """Offensive strategy: focus on maximizing own mobility"""
        return 15 * (len(playerMoves) - len(opponentMoves))

    def defensive(self, playerMoves, opponentMoves):
        """Defensive strategy: focus on minimizing opponent mobility"""
        return 10 * len(playerMoves) - 20 * len(opponentMoves)

    def offensiveToDefensive(self, playerMoves, opponentMoves, available_cells):
        """Early game, play an offensive strategy. Late game (<=12 cells), play a defensive strategy."""
        return self.offensive(playerMoves, opponentMoves) if available_cells > 12 else self.defensive(playerMoves, opponentMoves)

    # ── heurística ───────────────────────────────────────────────────────────

    def heuristic(self):
        """
        Heurística con estrategia ofensiva-defensiva que cambia según casillas disponibles
        """
        board = self._board()
        pos1  = self._pos1()
        pos2  = self._pos2()

        moves_max = self._valid_moves(pos1, board, pos2)
        moves_min = self._valid_moves(pos2, board, pos1)
        n_max = len(moves_max)
        n_min = len(moves_min)

        # ── Terminales ──
        if n_max == 0 and n_min == 0:
            return 0
        if n_max == 0:
            return -200
        if n_min == 0:
            return 200

        # Calculate available cells for strategy decision
        current_cells = sum(sum(row) for row in board)

        # Use offensive-to-defensive strategy based on available cells
        strategy_score = self.offensiveToDefensive(moves_max, moves_min, current_cells)

        # Add center control bonus
        center = 2.0
        dist_max = abs(pos1[0]-center) + abs(pos1[1]-center)
        dist_min = abs(pos2[0]-center) + abs(pos2[1]-center)
        centre_score = 2 * (dist_min - dist_max)

        return strategy_score + centre_score

    # ── auxiliares ───────────────────────────────────────────────────────────

    def cost(self):
        return self.level

    def __repr__(self):
        board = self._board()
        pos1, pos2 = self._pos1(), self._pos2()
        rows = ["   0 1 2 3 4", "  -----------"]
        for r in range(5):
            row_str = f"{r}| "
            for c in range(5):
                if   (r,c) == pos1:    row_str += 'X '
                elif (r,c) == pos2:    row_str += 'O '
                elif board[r][c] == 0: row_str += '# '
                else:                  row_str += '. '
            rows.append(row_str)
        return '\n'.join(rows)

In [32]:
def print_board(state):
    pos1, pos2, board = state['pos1'], state['pos2'], state['board']
    print("   0 1 2 3 4")
    print("  -----------")
    for r in range(5):
        row_str = f"{r}| "
        for c in range(5):
            if   (r,c) == pos1:    row_str += 'X '
            elif (r,c) == pos2:    row_str += 'O '
            elif board[r][c] == 0: row_str += '# '
            else:                  row_str += '. '
        print(row_str)
    print()


def get_valid_moves(pos, board, other_pos=None):
    """Obtiene movimientos válidos evitando superposición con la otra ficha"""
    MOVES = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
    r, c = pos
    return [
        (r+dr, c+dc)
        for dr, dc in MOVES
        if (0 <= r+dr < 5 and 0 <= c+dc < 5 and
            board[r+dr][c+dc] == 1 and
            (other_pos is None or (r+dr, c+dc) != other_pos))
    ]


def get_removable_cells(board, pos1, pos2):
    return [
        (r, c)
        for r in range(5) for c in range(5)
        if board[r][c] == 1 and (r,c) != pos1 and (r,c) != pos2
    ]


# ──────────────────────────────────────────────────────────────────────────────
#  Bucle principal
# ──────────────────────────────────────────────────────────────────────────────

def play_isolation():
    init_board = [[1]*5 for _ in range(5)]
    state = {
        'board':    init_board,
        'pos1':     (0, 2),    # X = Humano = Max
        'pos2':     (4, 2),    # O = IA     = Min
        'max_turn': False,      # el humano empieza
    }

    operators_isolation = ['mover+eliminar']
    DEPTH = 2   # Aumenta para IA más fuerte (más lento)

    human_turn = False

    print("=== ISOLATION 5×5 ===")
    print("Tú eres X (Max), la IA es O (Min)")
    print("# = casilla eliminada  |  Movimiento tipo rey (8 dir)")
    print("✅ FIXED: Las fichas ya NO se pueden superponer")
    print("📊 Se mostrará la cantidad de nodos explorados por la IA\n")

    while True:
        print_board(state)
        board = state['board']
        pos1, pos2 = state['pos1'], state['pos2']

        # ── TURNO HUMANO ────────────────────────────────────────────────────
        if human_turn:
            valid_moves = get_valid_moves(pos1, board, pos2)
            if not valid_moves:
                print("❌ No tienes movimientos. ¡Perdiste!")
                break

            print(f"Tu posición (X): {pos1}")
            print(f"Movimientos válidos: {valid_moves}")

            while True:
                try:
                    r = int(input("Mover a fila: "))
                    c = int(input("Mover a col:  "))
                    if (r, c) in valid_moves:
                        break
                    print("Movimiento inválido, intenta de nuevo.")
                except ValueError:
                    print("Ingresa números válidos.")

            state['pos1'] = (r, c)
            print_board(state)

            removable = get_removable_cells(state['board'], state['pos1'], state['pos2'])
            if removable:
                print(f"Casillas eliminables: {removable}")
                while True:
                    try:
                        r2 = int(input("Eliminar fila: "))
                        c2 = int(input("Eliminar col:  "))
                        if (r2, c2) in removable:
                            break
                        print("Casilla inválida, intenta de nuevo.")
                    except ValueError:
                        print("Ingresa números válidos.")
                state['board'][r2][c2] = 0
            else:
                print("(Sin casillas que eliminar)")

            state['max_turn'] = False
            human_turn = False

        # ── TURNO IA ─────────────────────────────────────────────────────────
        else:
            valid_moves_ai = get_valid_moves(pos2, board, pos1)
            if not valid_moves_ai:
                print("🎉 ¡La IA no tiene movimientos! ¡Ganaste!")
                break

            # Mostrar estrategia actual de la IA
            current_cells = sum(sum(row) for row in board)
            strategy = "OFENSIVA" if current_cells > 12 else "DEFENSIVA"
            print(f"🤖 IA usando estrategia {strategy} ({current_cells} casillas disponibles)")
            print("🤖 La IA está pensando…")

            import time
            start_time = time.time()

            ia_state = copy.deepcopy(state)
            ia_state['max_turn'] = False   # es turno de Min (IA)

            root = NodeIsolation(
                state=ia_state,
                value='root',
                operators=operators_isolation,
                player=False,
            )
            tree = Tree(root=root, operators=operators_isolation)
            best_child = tree.alphaBeta(depth=DEPTH)

            end_time = time.time()

            # Mostrar métricas de rendimiento
            print(f"📊 Nodos explorados: {tree.nodes_explored}")
            print(f"⏱️ Tiempo de procesamiento: {end_time - start_time:.3f} segundos")

            new_state = copy.deepcopy(best_child.state)

            # Informar qué hizo la IA
            old_pos2 = state['pos2']
            new_pos2 = new_state['pos2']
            print(f"🤖 IA movió O: {old_pos2} → {new_pos2}")

            old_board = state['board']
            new_board = new_state['board']
            for r in range(5):
                for c in range(5):
                    if old_board[r][c] == 1 and new_board[r][c] == 0 \
                       and (r,c) != old_pos2:
                        print(f"🤖 IA eliminó la casilla: ({r}, {c})")

            new_state['max_turn'] = True
            state = new_state
            human_turn = True
            print()  # Línea en blanco para mejor legibilidad

In [33]:
# Primero vamos a probar la carga de imágenes
import pygame
import os

def test_image_loading():
    """Prueba la carga de imágenes"""
    pygame.init()
    
    assets_dir = "game_assets"
    image_files = ["tableroRojo.png", "tableroAzul.png", "jugador.png", "IA.png"]
    
    print("🔍 PROBANDO CARGA DE IMÁGENES:")
    print(f"📂 Directorio actual: {os.getcwd()}")
    print(f"📂 Directorio assets: {assets_dir}")
    print()
    
    for img_file in image_files:
        img_path = os.path.join(assets_dir, img_file)
        print(f"📋 Probando: {img_path}")
        
        if os.path.exists(img_path):
            print(f"  ✅ Archivo existe")
            try:
                img = pygame.image.load(img_path)
                print(f"  ✅ Imagen cargada exitosamente: {img.get_width()}x{img.get_height()}")
            except Exception as e:
                print(f"  ❌ Error cargando imagen: {e}")
        else:
            print(f"  ❌ Archivo no existe")
        print()

# Ejecutar prueba
test_image_loading()

# Luego ejecutar el juego
print("=" * 50)
print("🎮 INICIANDO JUEGO:")
print("=" * 50)

🔍 PROBANDO CARGA DE IMÁGENES:
📂 Directorio actual: c:\Users\camil\Downloads\Ciberdefensa\Semestre 9\IA
📂 Directorio assets: game_assets

📋 Probando: game_assets\tableroRojo.png
  ✅ Archivo existe
  ✅ Imagen cargada exitosamente: 1024x1024

📋 Probando: game_assets\tableroAzul.png
  ✅ Archivo existe
  ✅ Imagen cargada exitosamente: 1024x1024

📋 Probando: game_assets\jugador.png
  ✅ Archivo existe
  ✅ Imagen cargada exitosamente: 111x104

📋 Probando: game_assets\IA.png
  ✅ Archivo existe
  ✅ Imagen cargada exitosamente: 100x101

🎮 INICIANDO JUEGO:


In [34]:
import pygame
import sys
import threading
import time
import os

class OptimizedIsolationGameGUI:
    def __init__(self, width=600, height=700):
        pygame.init()

        # Configuración de la pantalla
        self.width = width
        self.height = height
        self.screen = pygame.display.set_mode((width, height))
        pygame.display.set_caption("Isolation Game - Custom Graphics")

        # Configuración del tablero
        self.board_size = 5
        self.cell_size = 80
        self.board_offset_x = (width - self.board_size * self.cell_size) // 2
        self.board_offset_y = 100

        # Cargar imágenes reales
        self.load_custom_images()

        # Fuente para texto
        self.font = pygame.font.Font(None, 36)
        self.small_font = pygame.font.Font(None, 24)
        self.tiny_font = pygame.font.Font(None, 20)

        # Colores
        self.WHITE = (255, 255, 255)
        self.BLACK = (0, 0, 0)
        self.GREEN = (0, 255, 0)
        self.RED = (255, 0, 0)
        self.BLUE = (0, 0, 255)
        self.LIGHT_GRAY = (200, 200, 200)
        self.YELLOW = (255, 255, 0)
        self.DARK_GRAY = (100, 100, 100)

        # Estado del juego
        self.reset_game()

        # Variables de control
        self.selected_pos = None
        self.valid_moves = []
        self.game_phase = "move"  # "move" o "remove"
        self.removable_cells = []
        self.ai_thinking = False
        
        # Métricas de rendimiento de la IA
        self.last_nodes_explored = 0
        self.last_processing_time = 0.0

    def load_custom_images(self):
        """Carga las imágenes personalizadas del directorio game_assets"""
        print(f"🔍 Directorio de trabajo: {os.getcwd()}")
        
        # Verificar si el directorio existe
        assets_dir = "game_assets"
        if not os.path.exists(assets_dir):
            print(f"❌ No existe el directorio {assets_dir}")
            self.use_fallback_graphics()
            return
            
        try:
            # Cargar fichas de jugadores directamente (más simple)
            jugador_path = os.path.join(assets_dir, "jugador.png")
            ia_path = os.path.join(assets_dir, "IA.png")
            
            print(f"📂 Cargando imágenes desde {assets_dir}...")
            
            # Cargar fichas
            if os.path.exists(jugador_path):
                try:
                    player_original = pygame.image.load(jugador_path)
                    self.player1_piece = pygame.transform.scale(player_original, (self.cell_size, self.cell_size))
                    print("✅ Ficha del jugador (púrpura) cargada exitosamente")
                except Exception as e:
                    print(f"❌ Error cargando ficha jugador: {e}")
                    self.create_fallback_player1_piece()
            else:
                print(f"❌ No se encontró {jugador_path}")
                self.create_fallback_player1_piece()
                
            if os.path.exists(ia_path):
                try:
                    ia_original = pygame.image.load(ia_path)
                    self.player2_piece = pygame.transform.scale(ia_original, (self.cell_size, self.cell_size))
                    print("✅ Ficha de la IA (naranja) cargada exitosamente")
                except Exception as e:
                    print(f"❌ Error cargando ficha IA: {e}")
                    self.create_fallback_player2_piece()
            else:
                print(f"❌ No se encontró {ia_path}")
                self.create_fallback_player2_piece()
            
            # Para los tableros, usar un enfoque más simple
            tablero_rojo_path = os.path.join(assets_dir, "tableroRojo.png")
            tablero_azul_path = os.path.join(assets_dir, "tableroAzul.png")
            
            # Cargar tableros y usar una sección central
            if os.path.exists(tablero_rojo_path):
                try:
                    board_red_full = pygame.image.load(tablero_rojo_path)
                    # Tomar el centro de la imagen y redimensionar
                    w, h = board_red_full.get_size()
                    center_x, center_y = w // 2, h // 2
                    cell_extract_size = min(w // 8, h // 8)  # Tomar 1/8 del tamaño total
                    
                    # Extraer región central
                    extract_rect = pygame.Rect(
                        center_x - cell_extract_size // 2,
                        center_y - cell_extract_size // 2,
                        cell_extract_size,
                        cell_extract_size
                    )
                    
                    try:
                        cell_surface = board_red_full.subsurface(extract_rect)
                        self.board_red_image = pygame.transform.scale(cell_surface, (self.cell_size, self.cell_size))
                        print("✅ Tablero rojo cargado exitosamente")
                    except:
                        # Si subsurface falla, usar la imagen completa redimensionada
                        self.board_red_image = pygame.transform.scale(board_red_full, (self.cell_size, self.cell_size))
                        print("✅ Tablero rojo cargado (imagen completa redimensionada)")
                        
                except Exception as e:
                    print(f"❌ Error cargando tablero rojo: {e}")
                    self.create_fallback_red_board()
            else:
                print(f"❌ No se encontró {tablero_rojo_path}")
                self.create_fallback_red_board()
                
            if os.path.exists(tablero_azul_path):
                try:
                    board_blue_full = pygame.image.load(tablero_azul_path)
                    # Tomar el centro de la imagen y redimensionar
                    w, h = board_blue_full.get_size()
                    center_x, center_y = w // 2, h // 2
                    cell_extract_size = min(w // 8, h // 8)
                    
                    # Extraer región central
                    extract_rect = pygame.Rect(
                        center_x - cell_extract_size // 2,
                        center_y - cell_extract_size // 2,
                        cell_extract_size,
                        cell_extract_size
                    )
                    
                    try:
                        cell_surface = board_blue_full.subsurface(extract_rect)
                        self.board_blue_image = pygame.transform.scale(cell_surface, (self.cell_size, self.cell_size))
                        print("✅ Tablero azul cargado exitosamente")
                    except:
                        # Si subsurface falla, usar la imagen completa redimensionada
                        self.board_blue_image = pygame.transform.scale(board_blue_full, (self.cell_size, self.cell_size))
                        print("✅ Tablero azul cargado (imagen completa redimensionada)")
                        
                except Exception as e:
                    print(f"❌ Error cargando tablero azul: {e}")
                    self.create_fallback_blue_board()
            else:
                print(f"❌ No se encontró {tablero_azul_path}")
                self.create_fallback_blue_board()
            
            # Crear imagen para celdas bloqueadas (negro sin bordes)
            self.blocked_cell = pygame.Surface((self.cell_size, self.cell_size))
            self.blocked_cell.fill((20, 20, 20))  # Negro sólido, sin bordes
            
            # Crear superficies para estados especiales (sin bordes)
            self.valid_move_overlay = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
            self.valid_move_overlay.fill((200, 200, 200, 120))  # Gris claro semi-transparente
            
            self.removable_overlay = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
            self.removable_overlay.fill((255, 255, 0, 120))  # Amarillo semi-transparente
            
            print("🎨 Carga de imágenes completada sin bordes confusos")
            
        except Exception as e:
            print(f"❌ Error general cargando imágenes: {e}")
            print("🔄 Usando gráficos de respaldo...")
            self.use_fallback_graphics()

    def create_fallback_red_board(self):
        """Crea tablero rojo de respaldo sin bordes"""
        self.board_red_image = pygame.Surface((self.cell_size, self.cell_size))
        self.board_red_image.fill((150, 50, 50))  # Solo color sólido, sin bordes

    def create_fallback_blue_board(self):
        """Crea tablero azul de respaldo sin bordes"""
        self.board_blue_image = pygame.Surface((self.cell_size, self.cell_size))
        self.board_blue_image.fill((50, 50, 150))  # Solo color sólido, sin bordes

    def create_fallback_player1_piece(self):
        """Crea ficha púrpura de respaldo"""
        self.player1_piece = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
        center = self.cell_size // 2
        pygame.draw.circle(self.player1_piece, (200, 100, 255), (center, center), 30)
        pygame.draw.circle(self.player1_piece, (150, 50, 200), (center, center), 25)

    def create_fallback_player2_piece(self):
        """Crea ficha naranja de respaldo"""
        self.player2_piece = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
        center = self.cell_size // 2
        pygame.draw.circle(self.player2_piece, (255, 150, 50), (center, center), 30)
        pygame.draw.circle(self.player2_piece, (200, 100, 20), (center, center), 25)

    def use_fallback_graphics(self):
        """Usa gráficos simples sin bordes confusos"""
        print("🔄 Creando gráficos de respaldo sin bordes...")
        self.create_fallback_red_board()
        self.create_fallback_blue_board()
        self.create_fallback_player1_piece()
        self.create_fallback_player2_piece()
        
        # Negro sólido sin bordes para celdas bloqueadas
        self.blocked_cell = pygame.Surface((self.cell_size, self.cell_size))
        self.blocked_cell.fill((40, 40, 40))
        
        # Overlays sin bordes
        self.valid_move_overlay = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
        self.valid_move_overlay.fill((200, 200, 200, 100))
        
        self.removable_overlay = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
        self.removable_overlay.fill((255, 255, 0, 100))

    def reset_game(self):
        """Reinicia el juego"""
        self.game_state = {
            'board': [[1]*5 for _ in range(5)],
            'pos1': (0, 2),  # Jugador (X) - Púrpura
            'pos2': (4, 2),  # IA (O) - Naranja
            'max_turn': True  # True = turno del jugador
        }
        self.game_over = False
        self.winner = None
        self.human_turn = True
        self.selected_pos = None
        self.game_phase = "move"
        self.last_nodes_explored = 0
        self.last_processing_time = 0.0

    def get_valid_moves(self, pos, board, other_pos=None):
        """Obtiene movimientos válidos evitando superposición con la otra ficha"""
        MOVES = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
        r, c = pos
        return [
            (r+dr, c+dc)
            for dr, dc in MOVES
            if (0 <= r+dr < 5 and 0 <= c+dc < 5 and
                board[r+dr][c+dc] == 1 and
                (other_pos is None or (r+dr, c+dc) != other_pos))
        ]

    def get_removable_cells(self, board, pos1, pos2):
        """Obtiene celdas que se pueden eliminar"""
        return [
            (r, c)
            for r in range(5) for c in range(5)
            if board[r][c] == 1 and (r,c) != pos1 and (r,c) != pos2
        ]

    def screen_to_board(self, screen_pos):
        """Convierte coordenadas de pantalla a coordenadas del tablero"""
        x, y = screen_pos
        board_x = (x - self.board_offset_x) // self.cell_size
        board_y = (y - self.board_offset_y) // self.cell_size

        if 0 <= board_x < 5 and 0 <= board_y < 5:
            return (board_y, board_x)  # fila, columna
        return None

    def board_to_screen(self, board_pos):
        """Convierte coordenadas del tablero a coordenadas de pantalla"""
        r, c = board_pos
        x = self.board_offset_x + c * self.cell_size
        y = self.board_offset_y + r * self.cell_size
        return (x, y)

    def draw_board(self):
        """Dibuja el tablero usando las imágenes personalizadas SIN BORDES"""
        board = self.game_state['board']
        pos1 = self.game_state['pos1']
        pos2 = self.game_state['pos2']

        # Determinar qué tablero usar según casillas disponibles (≤12 = cambio)
        current_cells = sum(sum(row) for row in board)
        
        # Usar tablero rojo cuando >12 casillas, azul cuando ≤12
        board_image = self.board_red_image if current_cells > 12 else self.board_blue_image

        for r in range(5):
            for c in range(5):
                x, y = self.board_to_screen((r, c))

                if board[r][c] == 0:  # Celda bloqueada - negro sólido
                    self.screen.blit(self.blocked_cell, (x, y))
                else:
                    # Dibujar celda base del tablero (tus imágenes personalizadas)
                    self.screen.blit(board_image, (x, y))
                    
                    # Overlays para movimientos válidos o celdas eliminables
                    if (r, c) in self.valid_moves:
                        self.screen.blit(self.valid_move_overlay, (x, y))
                    elif (r, c) in self.removable_cells:
                        self.screen.blit(self.removable_overlay, (x, y))

                # Dibujar fichas de los jugadores (tus imágenes personalizadas)
                if (r, c) == pos1:  # Jugador humano - ficha púrpura
                    self.screen.blit(self.player1_piece, (x, y))
                elif (r, c) == pos2:  # IA - ficha naranja
                    self.screen.blit(self.player2_piece, (x, y))

    def draw_ui(self):
        """Dibuja la interfaz de usuario"""
        # Título
        title = self.font.render("ISOLATION - Custom Graphics", True, self.BLACK)
        title_rect = title.get_rect(center=(self.width//2, 30))
        self.screen.blit(title, title_rect)

        # Estado del turno
        if self.game_over:
            if self.winner == "human":
                status = "¡GANASTE! 🎉"
                color = self.GREEN
            elif self.winner == "ai":
                status = "IA GANÓ 🤖"
                color = self.RED
            else:
                status = "EMPATE"
                color = self.BLACK
        elif self.ai_thinking:
            status = "IA pensando... (Depth 3)"
            color = self.RED
        elif self.human_turn:
            if self.game_phase == "move":
                status = "Tu turno - Selecciona movimiento (Ficha Púrpura)"
            else:
                status = "Tu turno - Elimina una celda"
            color = (150, 50, 200)  # Púrpura
        else:
            status = "Turno de la IA (Ficha Naranja)"
            color = (200, 100, 50)  # Naranja

        status_text = self.small_font.render(status, True, color)
        status_rect = status_text.get_rect(center=(self.width//2, 60))
        self.screen.blit(status_text, status_rect)

        # Mostrar estrategia actual de la IA basada en casillas disponibles
        if not self.game_over:
            current_cells = sum(sum(row) for row in self.game_state['board'])
            strategy = "OFENSIVA" if current_cells > 12 else "DEFENSIVA"
            board_color = "ROJO" if current_cells > 12 else "AZUL"
            
            strategy_text = self.small_font.render(
                f"IA: {current_cells} casillas → {strategy} | Tablero {board_color}", 
                True, self.BLACK
            )
            strategy_rect = strategy_text.get_rect(center=(self.width//2, 80))
            self.screen.blit(strategy_text, strategy_rect)

        # Mostrar métricas de rendimiento de la IA
        if self.last_nodes_explored > 0:
            metrics_text = self.tiny_font.render(
                f"📊 Último movimiento IA: {self.last_nodes_explored} nodos | {self.last_processing_time:.3f}s", 
                True, self.DARK_GRAY
            )
            metrics_rect = metrics_text.get_rect(center=(self.width//2, self.height - 50))
            self.screen.blit(metrics_text, metrics_rect)

        # Instrucciones
        if not self.game_over:
            if self.human_turn and self.game_phase == "move":
                instructions = ["Haz clic en una casilla resaltada para mover"]
            elif self.human_turn and self.game_phase == "remove":
                instructions = ["Haz clic en una casilla amarilla para eliminar"]
            else:
                instructions = [""]
        else:
            instructions = ["Presiona R para reiniciar", "Presiona ESC para salir"]

        y_offset = self.board_offset_y + 5 * self.cell_size + 30
        for i, instruction in enumerate(instructions):
            if instruction:
                inst_text = self.small_font.render(instruction, True, self.BLACK)
                inst_rect = inst_text.get_rect(center=(self.width//2, y_offset + i * 25))
                self.screen.blit(inst_text, inst_rect)

    def handle_human_move(self, board_pos):
        """Maneja el movimiento del jugador humano"""
        if self.game_phase == "move":
            if board_pos in self.valid_moves:
                # Realizar movimiento
                self.game_state['pos1'] = board_pos

                # Verificar si hay celdas para eliminar
                self.removable_cells = self.get_removable_cells(
                    self.game_state['board'],
                    self.game_state['pos1'],
                    self.game_state['pos2']
                )

                if self.removable_cells:
                    self.game_phase = "remove"
                else:
                    # No hay celdas para eliminar, terminar turno
                    self.end_human_turn()

        elif self.game_phase == "remove":
            if board_pos in self.removable_cells:
                # Eliminar celda
                r, c = board_pos
                self.game_state['board'][r][c] = 0
                self.end_human_turn()

    def end_human_turn(self):
        """Termina el turno del jugador humano"""
        self.game_state['max_turn'] = False
        self.human_turn = False
        self.game_phase = "move"
        self.valid_moves = []
        self.removable_cells = []

    def ai_move(self):
        """Ejecuta el movimiento de la IA usando algoritmo estándar con depth 3"""
        self.ai_thinking = True

        # Verificar si la IA tiene movimientos válidos (sin superposición)
        pos2 = self.game_state['pos2']
        pos1 = self.game_state['pos1']
        board = self.game_state['board']
        valid_moves_ai = self.get_valid_moves(pos2, board, pos1)

        if not valid_moves_ai:
            self.game_over = True
            self.winner = "human"
            self.ai_thinking = False
            return

        # Usar el algoritmo estándar alpha-beta con depth 3
        ia_state = copy.deepcopy(self.game_state)
        ia_state['max_turn'] = False

        root = NodeIsolation(
            state=ia_state,
            value='root',
            operators=['mover+eliminar'],
            player=False,
        )

        # Medir tiempo y nodos explorados
        start_time = time.time()
        tree = Tree(root=root, operators=['mover+eliminar'])
        best_child = tree.alphaBeta(depth=3)
        end_time = time.time()

        # Guardar métricas para mostrar en pantalla
        self.last_nodes_explored = tree.nodes_explored
        self.last_processing_time = end_time - start_time

        if best_child:
            new_state = copy.deepcopy(best_child.state)
            self.game_state = {
                'board': new_state['board'],
                'pos1': new_state['pos1'],
                'pos2': new_state['pos2'],
                'max_turn': True
            }

        self.human_turn = True
        self.ai_thinking = False

        # Verificar si el jugador tiene movimientos válidos (sin superposición)
        pos1_new = self.game_state['pos1']
        pos2_new = self.game_state['pos2']
        valid_moves_human = self.get_valid_moves(pos1_new, self.game_state['board'], pos2_new)

        if not valid_moves_human:
            self.game_over = True
            self.winner = "ai"

    def update_valid_moves(self):
        """Actualiza los movimientos válidos del jugador evitando superposición"""
        if self.human_turn and self.game_phase == "move" and not self.game_over:
            pos1 = self.game_state['pos1']
            pos2 = self.game_state['pos2']
            self.valid_moves = self.get_valid_moves(pos1, self.game_state['board'], pos2)
            if not self.valid_moves:
                self.game_over = True
                self.winner = "ai"

    def run(self):
        """Bucle principal del juego optimizado"""
        clock = pygame.time.Clock()
        running = True
        
        print("🎮 Iniciando juego con gráficos personalizados...")
        print("🟣 Ficha púrpura = Jugador Humano")
        print("🟠 Ficha naranja = IA")
        print("✅ FIXED: Las fichas YA NO se pueden superponer")
        print("📊 Cambio de estrategia cuando hay ≤12 casillas disponibles")
        print("🎨 Usando tus imágenes personalizadas SIN BORDES CONFUSOS")
        print("🧠 IA usando algoritmo estándar Alpha-Beta con DEPTH 3")
        print("⭐ Tablero cambia de ROJO (>12) a AZUL (≤12)")
        print("📊 Se mostrarán métricas de rendimiento en pantalla")

        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r and self.game_over:
                        self.reset_game()
                    elif event.key == pygame.K_ESCAPE:
                        running = False

                elif event.type == pygame.MOUSEBUTTONDOWN:
                    if event.button == 1:  # Clic izquierdo
                        board_pos = self.screen_to_board(event.pos)
                        if board_pos and self.human_turn and not self.game_over:
                            self.handle_human_move(board_pos)

            # Actualizar estado del juego
            if not self.game_over:
                if self.human_turn:
                    self.update_valid_moves()
                elif not self.ai_thinking:
                    # Ejecutar movimiento de IA en un hilo separado
                    threading.Thread(target=self.ai_move, daemon=True).start()

            # Dibujar todo (usando tus imágenes personalizadas SIN BORDES)
            self.screen.fill(self.WHITE)
            self.draw_board()
            self.draw_ui()

            pygame.display.flip()
            clock.tick(60)  # 60 FPS para fluidez

        pygame.quit()

# Función para ejecutar el juego con imágenes personalizadas
def play_isolation_gui_optimized():
    """Inicia el juego con tus gráficos personalizados y algoritmo estándar depth 3"""
    try:
        game = OptimizedIsolationGameGUI()
        game.run()
    except Exception as e:
        print(f"❌ Error ejecutando el juego: {e}")
        print("Asegúrate de que pygame esté instalado correctamente")

In [35]:
play_isolation_gui_optimized()

🔍 Directorio de trabajo: c:\Users\camil\Downloads\Ciberdefensa\Semestre 9\IA
📂 Cargando imágenes desde game_assets...
✅ Ficha del jugador (púrpura) cargada exitosamente
✅ Ficha de la IA (naranja) cargada exitosamente
✅ Tablero rojo cargado exitosamente
✅ Tablero azul cargado exitosamente
🎨 Carga de imágenes completada sin bordes confusos
🎮 Iniciando juego con gráficos personalizados...
🟣 Ficha púrpura = Jugador Humano
🟠 Ficha naranja = IA
✅ FIXED: Las fichas YA NO se pueden superponer
📊 Cambio de estrategia cuando hay ≤12 casillas disponibles
🎨 Usando tus imágenes personalizadas SIN BORDES CONFUSOS
🧠 IA usando algoritmo estándar Alpha-Beta con DEPTH 3
⭐ Tablero cambia de ROJO (>12) a AZUL (≤12)
📊 Se mostrarán métricas de rendimiento en pantalla
